In [1]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import os
from sklearn.metrics import mean_squared_error

In [2]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [3]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
t = test_data[test_data['SoH'] != -100]
d = dev_data[dev_data['SoH'] != -100]
combined_df = pd.concat([t, d])
train_data=pd.concat([train_data,combined_df])
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)
#训练集y
train_y=train_data['SoH']
#填补缺失值
train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())
train_data=train_data.drop(columns=['SoH'])
train_data=train_data.drop(columns=['CS_Name'])
# train_data['log_cycle_mul_capacity'] = train_data['log_cycle'] * train_data['log_cycle']

# # 选择要循环相乘的列
# col_to_multiply = 'log_cycle'
# # 获取除了 col_to_multiply 列之外的所有列
# cols_to_multiply_with = train_data.drop(col_to_multiply, axis=1).columns
# # 循环相乘
# for col in cols_to_multiply_with:
#     new_col_name = f"{col_to_multiply}_mul_{col}"
#     train_data[new_col_name] = train_data[col_to_multiply] * train_data[col]

# 选择要循环相乘的列
col_to_multiply = 'cycle'
# 获取除了 col_to_multiply 列之外的所有列
cols_to_multiply_with = train_data.drop(col_to_multiply, axis=1).columns
# 循环相乘
for col in cols_to_multiply_with:
    new_col_name = f"{col_to_multiply}_mul_{col}"
    train_data[new_col_name] = train_data[col_to_multiply] * train_data[col]
train_data['log_cycle'] = np.log(train_data['cycle'])

In [4]:
t_data = test_data[test_data['SoH'] == -100]
#填补缺失值
t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())
t_data=t_data.drop(columns=['SoH'])
t_data=t_data.drop(columns=['CS_Name'])

# 选择要循环相乘的列
col_to_multiply = 'cycle'
# 获取除了 col_to_multiply 列之外的所有列
cols_to_multiply_with = t_data.drop(col_to_multiply, axis=1).columns
# 循环相乘
for col in cols_to_multiply_with:
    new_col_name = f"{col_to_multiply}_mul_{col}"
    t_data[new_col_name] = t_data[col_to_multiply] * t_data[col]
t_data['log_cycle'] = np.log(t_data['cycle'])

C:\Users\yubin\AppData\Local\Temp\ipykernel_10872\2471385484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
C:\Users\yubin\AppData\Local\Temp\ipykernel_10872\2471385484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())


In [1]:

    # 建立XGB模型
model = xgb.XGBRegressor(
    n_estimators=1000,  # 迭代次数
    learning_rate=0.01,  # 学习率
    objective='reg:squarederror',  # 损失函数
    random_state=42
)


train_x=train_data;

t_data_x=t_data;

train_x.to_csv('train_x.csv', index=False)
train_y.to_csv('train_y.csv', index=False)
model.fit(train_x, train_y)
test_y = model.predict(t_data_x)


# rmse = np.sqrt(mean_squared_error(test_y, train_y))
# rmse

NameError: name 'xgb' is not defined

In [6]:
# 假设 test_y 是一个一维数组，你可以将其转换为 DataFrame 对象
test_y_df = pd.DataFrame({'result': test_y})
test_=raw_test_data[raw_test_data['SoH'] == -100]
test_ = test_.reset_index(drop=True)
# 按列拼接预测结果和 t_data_x
merged_df = pd.concat([test_, test_y_df],axis=1)
# merged_df.rename(columns={'Unamed: 0': 'first'}, inplace=True)
selected_columns = ['Unnamed: 0', 'cycle', 'CS_Name','result']
# 选择特定列
new_df = merged_df.loc[:, selected_columns]
new_df.to_csv('result.csv', index=False)
# # 保存拼接后的结果到新的 CSV 文件
# merged_df.to_csv('merged_result.csv', index=False)

In [41]:
new_df

,Unnamed: 0,cycle,CS_Name,result
0,105,735.0,CS2_35,0.479337
1,106,736.0,CS2_35,0.462309
2,107,737.0,CS2_35,0.453843
3,108,738.0,CS2_35,0.454750
4,109,739.0,CS2_35,0.435222
...,...,...,...,...
115,147,857.0,CS2_38,0.428448
116,148,858.0,CS2_38,0.426147
117,149,859.0,CS2_38,0.426869
118,150,860.0,CS2_38,0.425416


In [22]:
raw_test_data

,Unnamed: 0,cycle,capacity,SoH,resistance,CCCT,CVCT,CS_Name
0,0,630.0,0.870464,0.623609,0.093925,4762.384349,2565.172418,CS2_35
1,1,631.0,0.859885,0.614440,0.096180,4663.710466,2599.623561,CS2_35
2,2,632.0,0.833305,0.614332,0.099413,4636.225554,1628.032077,CS2_35
3,3,633.0,0.815995,0.614295,0.101120,0.000000,1236.437937,CS2_35
4,4,634.0,0.832306,0.614363,0.096528,4895.255376,2277.145610,CS2_35
...,...,...,...,...,...,...,...,...
570,147,857.0,0.614230,-100.000000,0.111245,3002.888352,3206.960871,CS2_38
571,148,858.0,0.605064,-100.000000,0.111345,2961.199236,3206.492017,CS2_38
572,149,859.0,0.605071,-100.000000,0.110633,2923.872113,3221.273252,CS2_38
573,150,860.0,0.605073,-100.000000,0.110533,2907.340689,3222.194994,CS2_38
